In [ ]:
!pip install optuna

Source: https://www.youtube.com/watch?v=QIUxPv5PJOY [13.11.2021]

Source: https://www.youtube.com/watch?v=vB8SleIKlcw [13.11.2021]

Source: https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/ [13.11.2021]

Source: https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/ [17.11.2021]

Source: https://medium.com/@Minyus86/optkeras-112bcc34ec73 [27.11.2021]

Source: https://github.com/optuna/optuna-examples/blob/main/keras/keras_simple.py [27.11.2021]

Source: https://stackoverflow.com/questions/33211988/python-reshape-3d-array-into-2d [02.12.2021]

Source: https://machinelearningmastery.com/check-point-deep-learning-models-keras/ [09.12.2021]

In [ ]:
import pandas as pd
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Flatten
from sklearn.metrics import mean_squared_error
import optuna
import keras.backend as K
from keras.callbacks import ModelCheckpoint
import keras

Load dataset

In [ ]:
df = pd.read_csv('drive/MyDrive/BA_JSON/input_data.csv')
df = df.rename(columns={'Unnamed: 0': 'tickCount'})

In [ ]:
df.head(5)

,tickCount,buttons,mouseDeltaX,mouseDeltaY
0,718,[],0,0
1,719,[],0,0
2,720,[],0,0
3,721,[],0,0
4,722,[],0,0


Remove Button input

In [ ]:
df = df.drop('buttons', axis=1)
df = df.set_index('tickCount')

In [ ]:
df.head(5)

,mouseDeltaX,mouseDeltaY
tickCount,,
718,0,0
719,0,0
720,0,0
721,0,0
722,0,0


In [ ]:
df[500000:500010]

,mouseDeltaX,mouseDeltaY
tickCount,,
6099189,0,0
6099190,0,0
6099191,5,0
6099192,16,-1
6099193,3,-4
6099194,-1,-10
6099195,-4,-5
6099196,-6,1
6099197,-11,11


Normalize the data

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(df)

scaled_data

array([[0.57908728, 0.52347826],
       [0.57908728, 0.52347826],
       [0.57908728, 0.52347826],
       ...,
       [0.57908728, 0.52347826],
       [0.57997342, 0.52347826],
       [0.57908728, 0.52347826]])

Creates the training data with the lookback

In [ ]:
def create_dataset(dataset, look_back):
  X_all, y_all = [], []
  for i in range(len(scaled_data) - look_back - 1):
    deltaX = scaled_data[i:(i + look_back), 0]
    deltaY = scaled_data[i:(i + look_back), 1]
    X_all.append([deltaX, deltaY])
    y_all.append(scaled_data[i + look_back])
  return np.array(X_all), np.array(y_all)

Creat Dataset with last N inputs

In [ ]:
look_back = 6
X_all, y_all = create_dataset(scaled_data, look_back)
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2)

In [ ]:
X_train[:5]

array([[[0.57908728, 0.57908728, 0.57908728, 0.57908728, 0.57953035,
         0.57908728],
        [0.52347826, 0.52347826, 0.52347826, 0.52347826, 0.52347826,
         0.52347826]],

       [[0.57908728, 0.57908728, 0.57908728, 0.57908728, 0.57908728,
         0.57908728],
        [0.52347826, 0.52347826, 0.52347826, 0.52347826, 0.52347826,
         0.52347826]],

       [[0.57509969, 0.57731502, 0.57908728, 0.57908728, 0.57908728,
         0.57908728],
        [0.53043478, 0.53043478, 0.52695652, 0.52347826, 0.52347826,
         0.52347826]],

       [[0.54674346, 0.5449712 , 0.5449712 , 0.5476296 , 0.55604785,
         0.55782012],
        [0.53391304, 0.53043478, 0.53391304, 0.5373913 , 0.53043478,
         0.52347826]],

       [[0.58218875, 0.58130261, 0.58174568, 0.58130261, 0.58085955,
         0.58085955],
        [0.52347826, 0.52347826, 0.52347826, 0.52347826, 0.52347826,
         0.52347826]]])

In [ ]:
y_train[:5]

array([[0.57908728, 0.52347826],
       [0.57908728, 0.52347826],
       [0.57997342, 0.52347826],
       [0.55516172, 0.52347826],
       [0.58085955, 0.52347826]])

Optuna

In [ ]:
def objective(trial):
  K.clear_session()

  lb = trial.suggest_int('lookback', 3, 15)

  X_all, y_all = create_dataset(scaled_data, look_back)
  X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2)

  model = Sequential()
  model.add(Dense(
      units = trial.suggest_int('units_layer_1', 1, 200)
  ))
  hidden_activation = trial.suggest_categorical('activation_layer_2', ['relu', 'sigmoid'])
  model.add(Dense(
      units = trial.suggest_int('units_layer_2', 1, 200),
      activation = hidden_activation
      ))
  layers_n = trial.suggest_int('layers_n', 1, 3)
  if(layers_n >= 2):
    model.add(Dense(
        units = trial.suggest_int('units_layer_3', 1, 200),
        activation = hidden_activation
    ))
  if(layers_n == 3):
    model.add(Dense(
        units = trial.suggest_int('units_layer_4', 1, 200),
        activation = hidden_activation
    ))
  model.add(Dense(
      units = 1, 
      activation = trial.suggest_categorical('activation_layer_out', ['linear', 'sigmoid'])
      ))
  model.compile(
      loss = 'mean_squared_error', 
      optimizer = trial.suggest_categorical('optimizer', ['adam', 'RMSprop']))
  model.fit(X_train, y_train, 
            epochs = 200, 
            batch_size = trial.suggest_int('batch_size', 128, 512),
            verbose=0)
  
  testPredict = model.predict(X_test)
  testPredict = testPredict.transpose(2,0,1).reshape(-1,testPredict.shape[1])
  testPredict = scaler.inverse_transform(testPredict)
  test = scaler.inverse_transform(y_test)
  score =  math.sqrt(mean_squared_error(test[:,0], testPredict[:,0]))
  score += math.sqrt(mean_squared_error(test[:,1], testPredict[:,1]))

  return score

In [ ]:
study =  optuna.create_study(direction='minimize')
study.optimize(objective, timeout=25000)
print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
  print("    {}: {}".format(key, value))

[I 2021-12-08 11:04:22,820] A new study created in memory with name: no-name-380a26b4-88df-462a-946a-c33ccec8c065
[I 2021-12-08 11:11:24,314] Trial 0 finished with value: 13.034521889436013 and parameters: {'lookback': 15, 'units_layer_1': 134, 'activation_layer_2': 'sigmoid', 'units_layer_2': 7, 'layers_n': 1, 'activation_layer_out': 'sigmoid', 'optimizer': 'RMSprop', 'batch_size': 502}. Best is trial 0 with value: 13.034521889436013.
[I 2021-12-08 11:26:44,231] Trial 1 finished with value: 13.267012384257416 and parameters: {'lookback': 15, 'units_layer_1': 30, 'activation_layer_2': 'relu', 'units_layer_2': 154, 'layers_n': 1, 'activation_layer_out': 'linear', 'optimizer': 'RMSprop', 'batch_size': 169}. Best is trial 0 with value: 13.034521889436013.
[I 2021-12-08 11:32:31,103] Trial 2 finished with value: 10.850494205946848 and parameters: {'lookback': 6, 'units_layer_1': 10, 'activation_layer_2': 'relu', 'units_layer_2': 58, 'layers_n': 1, 'activation_layer_out': 'sigmoid', 'optimi

Best trial (MSE=9.73473):
- lookback: 6
- units_layer_1: 46
- units_layer_2: 118 
- units_layer_3: 196 
- units_layer_4: 1
- activaiton_hidden_layer: relu
- activation_output_layer: sigmoid
- optimizer: adam
- batch_size: 355 

Train the model

In [ ]:
model = Sequential()
model.add(Dense(46))
model.add(Dense(units=118, activation='relu'))
model.add(Dense(units=196, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(loss='mean_squared_error', optimizer='adam')

filepath = 'drive/MyDrive/deltaNN/weights-improvement-{epoch:d}.hdf5' 
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

model.fit(X_train, y_train, validation_split=0.2, epochs=3000, batch_size=355, callbacks=callbacks_list, verbose=2)

In [ ]:
model.save('drive/MyDrive/deltaNN/3000.hdf5')

best val_loss: 0.00001

In [ ]:
best_val_model = Sequential()
best_val_model.add(Dense(46))
best_val_model.add(Dense(units=118, activation='relu'))
best_val_model.add(Dense(units=196, activation='relu'))
best_val_model.add(Dense(units=1, activation='sigmoid'))

best_val_model = keras.models.load_model('drive/MyDrive/deltaNN/weights-improvement-737.hdf5')

trainPredict = best_val_model.predict(X_train)
trainPredict = trainPredict.transpose(2,0,1).reshape(-1,trainPredict.shape[1])
testPredict = best_val_model.predict(X_test)
testPredict = testPredict.transpose(2,0,1).reshape(-1,testPredict.shape[1])

trainPredict = scaler.inverse_transform(trainPredict)
y_train = scaler.inverse_transform(y_train)
testPredict = scaler.inverse_transform(testPredict)
y_test = scaler.inverse_transform(y_test)

trainScore = math.sqrt(mean_squared_error(y_train[:,0], trainPredict[:,0]))
print('Train Score deltaX: %.2f RMSE' % (trainScore))
trainScore = math.sqrt(mean_squared_error(y_train[:,1], trainPredict[:,1]))
print('Train Score deltaY: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(y_test[:,0], testPredict[:,0]))
print('Test Score deltaX: %.2f RMSE' % (testScore))
testScore = math.sqrt(mean_squared_error(y_test[:,1], testPredict[:,1]))
print('Test Score deltaY: %.2f RMSE' % (testScore))

Train Score deltaX: 7.88 RMSE
Train Score deltaY: 1.93 RMSE
Test Score deltaX: 7.21 RMSE
Test Score deltaY: 1.96 RMSE


Train Score deltaX: 7.88 RMSE

Train Score deltaY: 1.93 RMSE

Test Score deltaX: 7.21 RMSE

Test Score deltaY: 1.96 RMSE

Look at some example predictions

In [ ]:
for i in range(20):
  print('Value:')
  print(y_test[i])
  print('Score:')
  print(testPredict[i])

Value:
[0. 0.]
Score:
[0.04541027 0.09637976]
Value:
[-23. -12.]
Score:
[-14.859584   -6.2124996]
Value:
[0. 0.]
Score:
[0.04541027 0.09637976]
Value:
[0. 0.]
Score:
[0.22782981 0.13935769]
Value:
[ 0. -1.]
Score:
[1.929336   0.08020306]
Value:
[ 0. -1.]
Score:
[-1.059331  -0.3477255]
Value:
[-2.  4.]
Score:
[-3.95921    3.8037918]
Value:
[0. 0.]
Score:
[0.6499777  0.09637976]
Value:
[-1.  0.]
Score:
[-0.9502291   0.09637976]
Value:
[-2.  0.]
Score:
[0.04541027 0.18302107]
Value:
[0. 0.]
Score:
[0.04541027 0.09637976]
Value:
[65. 12.]
Score:
[79.84562  14.816561]
Value:
[ 0. -4.]
Score:
[ 0.41764838 -3.1813555 ]
Value:
[0. 0.]
Score:
[0.04541027 0.09637976]
Value:
[0. 0.]
Score:
[0.04541027 0.09637976]
Value:
[0. 0.]
Score:
[0.04541027 0.09637976]
Value:
[3. 3.]
Score:
[2.7944834 0.1108771]
Value:
[-22.   3.]
Score:
[-33.31907     3.7734947]
Value:
[0. 0.]
Score:
[ 0.04541027 -0.27283973]
Value:
[0. 0.]
Score:
[0.04541027 0.09637976]


Source: https://stackoverflow.com/questions/1557571/how-do-i-get-time-of-a-python-programs-executionm[13.01.2022]

Calculate the time for one prediction

In [ ]:
import time
start_time = time.time()
best_val_model.predict(X_all)
duration = time.time() - start_time
print("--- %s seconds ---" % (duration))
print(duration/len(X_all))

--- 20.538424253463745 seconds ---
3.184498682605434e-05


In [ ]:
len(X_all)

644950

Time total: 20.538424253463745 seconds

Time one prediction: 3.184498682605434e-05 seconds